# Import libraries

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functions.filter import *
from functions.signature import *

# Import data

In [22]:
df_train = pd.read_json("../data/train/TRAIN.json", encoding='utf-8')
# Convert the name of the columns to uppercase
df_train.columns = map(str.upper, df_train.columns)
# Drop the null register if the PALABRAS_EMPLEO_TEXTO or CATEGORIA or SUBCATEGORIA is null
print("Before:" + str(df_train.shape))
df_train = df_train.dropna(subset=['PALABRAS_EMPLEO_TEXTO', 'CATEGORIA', 'SUBCATEGORIA'])
print("After:" + str(df_train.shape))
df_train['NUM_WORDS'] = df_train['PALABRAS_EMPLEO_TEXTO'].apply(lambda x: len(x.split(" ")) - 1)
# Shuffle the dataframe
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train.head(2)

Before:(48893, 6)
After:(48893, 6)


,ID_OFERTA,SUBCATEGORIA,CATEGORIA,PALABRAS_EMPLEO_TEXTO,ID_PUESTO_ESCO,ID_PUESTO_ESCO_ULL,NUM_WORDS
0,083bc773384769ba079884ec5778d9,Atención al cliente,Atención a clientes,DEPENDIENTE TIENDAS MODA INFANTIL EMPRESAS SEC...,1984,1984,23
1,5791a15ae248feb75908bdc5d5cea2,Estética y cosmética,"Profesiones, artes y oficios",PELUQUERO OFICIAL PELUQUERIA EXPERIENCIA PLANT...,1916,1916,17


# Undersampling

In [23]:
print("Before:" + str(df_train.shape))
# Delete occupations with more than 1000 offers and set to 1000
df_undersampling = drop_randomly_most_offers(df_train, maxOffers=1000, totalOffers=800)
print("After:" + str(df_undersampling.shape))
# df_undersampling.to_json("../data/train/TRAIN_filter_1000_offers.json", orient='records', force_ascii=False)

Before:(48893, 7)
After:(38165, 7)


# Delete by Num of Words

In [24]:
print("Before: ", df_train.shape[0])
# Delete all rows with less than 5 words
df_words = delete_offers_with_less_words(df_train, 5)
print("After: ", df_train.shape[0])
# df_words.to_json("../data/train/TRAIN_filter_5_words.json", orient='records', force_ascii=False)

Before:  48893
After:  48893


# Delete by signature

In [27]:
print("Before: ", df_train.shape[0])
diccionario_ocupaciones = create_diccionario_ocupaciones(df_train)
df_train['RELATIVE_SIGNATURE'] = get_offers_signature_relative(df_train, diccionario_ocupaciones, precision=2)
show_signature_by_occupation(df_train, 1607)
df_signature = delete_offers_same_occupation_by_signature(df_train, maxOffers=100, totalOffers=100, precision=2)
show_signature_by_occupation(df_signature, 1607)
print("After: ", df_signature.shape[0])
df_signature.to_json("../data/train/TRAIN_filter_100_signature.json", orient='records', force_ascii=False)

Before:  48893


After:  45957


# Delete by signature and undersampling

In [28]:
print("Before: ", df_train.shape[0])
diccionario_ocupaciones = create_diccionario_ocupaciones(df_train)
df_train['RELATIVE_SIGNATURE'] = get_offers_signature_relative(df_train, diccionario_ocupaciones, precision=4)
df_signature = delete_offers_same_occupation_by_signature(df_train, maxOffers=100, totalOffers=100, precision=4)
df_signature_undersampling = drop_randomly_most_offers(df_signature, maxOffers=400, totalOffers=400)
print("After: ", df_signature.shape[0])

Before:  48893
After:  47117


In [ ]:
df_signature.to_json("../data/train/TRAIN_filter_100_signature.json", orient='records', force_ascii=False)